In [2]:
#!pip install line_profiler
%load_ext line_profiler

In [3]:
import numpy as np
from skimage import data, color
from skimage.transform import resize
from concurrent.futures import ThreadPoolExecutor

In [4]:
imgs = np.uint8(data.lfw_subset()*255)

def res_skimage(imgs):
    new_size = (imgs[1].shape[0]//2, imgs[1].shape[1]//2)
    res_im = []
    for im in imgs:
        image_resized = resize(im, new_size, anti_aliasing=True)
        res_im.append(image_resized)
    return np.asarray(res_im)


In [8]:
# 測試資料
imgs = np.random.randint(0, 255, (100, 64, 64), dtype=np.uint8)

# 使用 %lprun 進行剖析
%lprun -f res_skimage res_skimage(imgs)

Timer unit: 1e-07 s

Total time: 0.0405753 s
File: C:\Users\USER\AppData\Local\Temp\ipykernel_1720\634147576.py
Function: res_skimage at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def res_skimage(imgs):
     4         1         69.0     69.0      0.0      new_size = (imgs[1].shape[0]//2, imgs[1].shape[1]//2)
     5         1          3.0      3.0      0.0      res_im = []
     6       101        790.0      7.8      0.2      for im in imgs:
     7       100     401396.0   4014.0     98.9          image_resized = resize(im, new_size, anti_aliasing=True)
     8       100        741.0      7.4      0.2          res_im.append(image_resized)
     9         1       2754.0   2754.0      0.7      return np.asarray(res_im)

In [9]:
import numpy as np
from skimage.transform import resize
from concurrent.futures import ThreadPoolExecutor

def res_skimage(imgs):
    new_size = (imgs[1].shape[0] // 2, imgs[1].shape[1] // 2)
    
    def process_image(im):
        return resize(im, new_size, anti_aliasing=True)

    with ThreadPoolExecutor() as executor:
        res_im = list(executor.map(process_image, imgs))

    return np.asarray(res_im)


In [10]:
%lprun -f res_skimage res_skimage(imgs)

Timer unit: 1e-07 s

Total time: 0.078305 s
File: C:\Users\USER\AppData\Local\Temp\ipykernel_1720\4173701576.py
Function: res_skimage at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def res_skimage(imgs):
     6                                               # 只計算一次 new_size
     7         1        105.0    105.0      0.0      new_size = (imgs[1].shape[0] // 2, imgs[1].shape[1] // 2)
     8                                               
     9                                               # 使用多線程並行處理圖像
    10         1          6.0      6.0      0.0      def process_image(im):
    11                                                   return resize(im, new_size, anti_aliasing=True)
    12                                           
    13         1        548.0    548.0      0.1      with ThreadPoolExecutor() as executor:
    14         1     778406.0 778406.0     99.4          res_im = list(executor.map(process_ima

In [ ]:
import multiprocessing as mp
import time

# 原始計算 Pi 的函數
def approximate_pi(n):
    pi_2 = 1
    nom, den = 2.0, 1.0
    for i in range(n):
        pi_2 *= nom / den
        if i % 2:
            nom += 2
        else:
            den += 2
    return 2 * pi_2

# 測試數據
nums = [1_822_725, 22_059_421, 32_374_695, 88_754_320, 97_162_66, 200_745_654]

# 並行執行函數
def parallel_execution(nums):
    with mp.Pool(processes=4) as pool:
        results = pool.map(approximate_pi, nums)
    return results

# 主函數
def main():
    # 順序執行
    start = time.time()
    sequential_results = [approximate_pi(n) for n in nums]
    sequential_time = time.time() - start
    print(f"Sequential execution time: {sequential_time:.2f} seconds")
    
    # 並行執行
    start = time.time()
    parallel_results = parallel_execution(nums)
    parallel_time = time.time() - start
    print(f"Parallel execution time: {parallel_time:.2f} seconds")
    
    # 效能提升
    improvement = sequential_time / parallel_time
    print(f"Speedup: {improvement:.2f}x")
    
    # 返回結果
    return sequential_results, parallel_results, sequential_time, parallel_time, improvement

# 在 Jupyter Notebook 中執行
if __name__ == "__main__":
    sequential_results, parallel_results, sequential_time, parallel_time, improvement = main()


Sequential execution time: 32.15 seconds


In [ ]:
from numba import jit
import time

@jit(nopython=True)
def approximate_pi_optimized(n):
    pi_2 = 1
    nom, den = 2.0, 1.0
    for i in range(n):
        pi_2 *= nom / den
        if i % 2:
            nom += 2
        else:
            den += 2
    return 2 * pi_2